In [50]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [51]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYzJmOGMxNzgtMzFkZS00NGZlLThjZmQtMDJhNTkwNjBmZDAyIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyI0WVBvNUFzeEZWSjFxckFEWmlvcXZIUlhlbnpGVk1xamNpcnR0djlRVW94ZyJdLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [52]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNTNkY2RiNWMtYTA4NS00ZGFkLThkNjUtNTc4ZWQ4NzhjMmU3IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyJDTEFRNFRaTTRmTEs2UlpjZ1ZjaFY0ODhES2c1dEsyOU5wZ0pHcnBiNFRUVyJdLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ==


In [53]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': 'c2f8c178-31de-44fe-8cfd-02a59060fd02',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['4YPo5AsxFVJ1qrADZioqvHRXenzFVMqjcirttv9QUoxg'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [54]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 52srtqQP3RKmNLKSbdyuzZQ8evddbTBgRYt8mB6kw5DjoRBrJSm6GJkqtE2u3xSe2t1mKhUNWa8mMet3QcwDd46q

Public key: 
 8JV21jX3S8GEmovotdSVQtc5khsckbn6K942zhpXv6Zb


**Sending a connection request to acapy**

In [55]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [56]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")

In [57]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [58]:
   
connectionRequestResponseUnpacked = \
    util.unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "19c41eba-067e-48f5-a056-7193e4f75282", "~thread": {"thid": '
 '"9fcad62c-6939-4aca-82b1-ad0d0e50ca11"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"3KQC09vdrQJzzaUB2A2OxRV_mqP2yGah1xcuLNBYGgSFkE5CCwak9tb3ceqUUudt1iQYruRWBHyHGpSxGPewCw==", '
 '"sig_data": '
 '"AAAAAF7Ho817IkRJRCI6ICI4TnlSQ1BDVkZQRzdadVdHMzgxNkZ3IiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjo4TnlSQ1BDVkZQRzdadVdHMzgxNkZ3IiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6OE55UkNQQ1ZGUEc3WnVXRzM4MTZGdyMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OjhOeVJDUENWRlBHN1p1V0czODE2RnciLCAicHVibGljS2V5QmFzZTU4IjogIjUyRzRZUWpnMlYxVmVvNUJMRzRVa2FpYkhSYVNZdW9WSEc0Ujl4UDJDOEU4In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [59]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': '8NyRCPCVFPG7ZuWG3816Fw',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw',
            'publicKey': [{'controller': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw',
                           'id': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw#1',
                           'publicKeyBase58': '52G4YQjg2V1Veo5BLG4UkaibHRaSYuoVHG4R9xP2C8E8',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw;indy',
                         'priority': 0,
                         'recipientKeys': ['52G4YQjg2V1Veo5BLG4UkaibHRaSYuoVHG4R9xP2C8E8'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [60]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw',
             'publicKey': [{'controller': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw',
                            'id': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw#1',
                            'publicKeyBase58': '52G4YQjg2V1Veo5BLG4UkaibHRaSYuoVHG4R9xP2C8E8',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:8NyRCPCVFPG7ZuWG3816Fw;indy',
                          'priority': 0,
                          'recipientKeys': ['52G4YQjg2V1Veo5BLG4UkaibHRaSYuoVHG4R9xP2C8E8'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'\xc9\xaf\xc2|\xb3\x19y_\xfdI\xb5\xa6\x84L\xff\x960O\xc5*

### FEATURE DISCOVERY

In [61]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
message = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=message)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '23f61f7f-9350-44f3-a613-d0fe147b01d1',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0'},
               {'pid': 'https://didcomm.org/action-menu/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0'},
               {'pid': 'https://didcomm.org/basicmessage/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-taa/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-presentation/0.1'},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'pid': 'https://didcomm.org/credential-presentation/0.1'},
               {'pid': 'https://didcomm.org/credential-issuance/0.1'},
       

In [62]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/add",
    payload="{amazing: 'payload'}"
)

encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '2fe4c870-e357-400a-9c94-25629666eab8',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/add',
 'hashid': '79ea90f829f21cbfc9a9697ca5d835f508e751745ef091f185138acdd2e22c1e',
 'payload': "{amazing: 'payload'}",
 '~thread': {'thid': 'dead9673-8287-468e-95fa-7ed2a37b4ed2'}}


In [65]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/get",
    hashid=responseDecoded['hashid']
)

encodedMessage = util.packMessage(message, connections[0])

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '06ca1f95-0d06-472a-8b2d-a75427499b0d',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records/get',
 'hashid': '79ea90f829f21cbfc9a9697ca5d835f508e751745ef091f185138acdd2e22c1e',
 'payload': "{amazing: 'payload'}",
 '~thread': {'thid': 'fd8cc313-5a32-4b3f-ab48-de6d6aca6e34'}}
